In [1]:
import pandas as pd

DATA = '/kaggle/input/liming-and-rhizoctonia-root-rot-of-soybean/h6v8j338ds-1/Dataset.xlsx'
SHEET_NAME = 'Data'

df = pd.read_excel(io=DATA, sheet_name=SHEET_NAME)

df.head()

,Plot,Field,Site,Block,Dose,PLH,INSH,BRA,TNOD,FNOD,PODS,GRAINS,WEIGHT,PLANTS,YIELD,INC,AUDPC
0,33,NIT,Patch,1,0,77.9,3.20,3.0,16.8,12.6,46.0,114.4,155.500000,6.333333,1865.057471,44.18,753.98
1,34,NIT,Patch,1,3,85.4,3.05,0.9,17.9,11.8,45.4,112.2,176.300000,13.333333,4059.829715,15.51,657.92
2,35,NIT,Patch,1,6,82.3,3.00,1.1,17.4,12.6,41.3,99.5,163.266667,12.222222,3253.086420,11.94,355.69
3,36,NIT,Patch,1,9,87.5,3.05,0.8,17.5,12.1,46.7,112.7,176.333333,12.555556,3829.118774,9.70,387.57
4,37,NIT,Patch,2,0,83.2,2.25,0.3,17.1,10.6,34.8,84.4,156.400000,7.666667,1819.003831,49.78,522.98


In [2]:
df.nunique()

Plot      128
Field       4
Site        2
Block       4
Dose        4
PLH        98
INSH       31
BRA        34
TNOD       46
FNOD       53
PODS      109
GRAINS    114
WEIGHT    125
PLANTS    110
YIELD     128
INC        66
AUDPC      66
dtype: int64

In [3]:
from plotly import express

express.histogram(data_frame=df, x='YIELD')